In [172]:
import numpy as np

class State:
    def __init__(self, my_actions=None, enemy_actions=None):
        self.my_actions = my_actions if my_actions is not None else []
        self.enemy_actions = enemy_actions if enemy_actions is not None else []
        self.current_player = 'me'  # 첫 번째 플레이어 나로 초기화
        self.board = self.create_board(self.my_actions, self.enemy_actions)

    def create_board(self, my_actions, enemy_actions):
        total_board = np.zeros(shape=(2, 3, 3))

        # 각 플레이어의 말이 놓인 위치를 1로 표시
        for action in my_actions:
            row, col = divmod(action, 3)
            total_board[0][row, col] = 1  # 나의 보드
        for action in enemy_actions:
            row, col = divmod(action, 3)
            total_board[1][row, col] = 1  # 상대방 보드

        return total_board

    def available_moves(self):
        # 나와 상대 모두 놓지 않은 빈 칸을 반환
        total_occupied = self.board[0] + self.board[1]
        available = np.argwhere(total_occupied == 0)
        return [tuple(pos) for pos in available]

    def next_state(self, action):
        new_state = State(self.my_actions.copy(), self.enemy_actions.copy())
        new_state.add_move(action)
        new_state.current_player = self.switch_player(self.current_player)  # 플레이어 전환
        return new_state

    def add_move(self, action):
        if self.current_player == 'me':
            self.my_actions.append(action[0] * 3 + action[1])  # 2D 좌표를 1D로 변환하여 저장
        else:
            self.enemy_actions.append(action[0] * 3 + action[1])

        # 보드 다시 생성
        self.board = self.create_board(self.my_actions, self.enemy_actions)

    def is_win(self, board):
        vertical_win = np.any(np.all(board == 1, axis=0))  # 세로 줄 확인
        horizontal_win = np.any(np.all(board == 1, axis=1))  # 가로 줄 확인
        diagonal_win1 = np.all(np.diag(board) == 1)  # 첫 번째 대각선 확인
        diagonal_win2 = np.all(np.diag(np.fliplr(board)) == 1)  # 두 번째 대각선 확인

        # 하나라도 승리 조건을 만족하면 True 반환
        return vertical_win or horizontal_win or diagonal_win1 or diagonal_win2

    def is_lose(self):
        return self.is_win(self.board[1])  # 상대방 보드에서 승리 여부 확인

    def is_draw(self):
        # 남은 칸이 없고, 나와 상대방 모두 승리하지 않은 경우 무승부
        return len(self.available_moves()) == 0 and not self.is_win(self.board[0]) and not self.is_win(self.board[1])

    def is_done(self):
        return self.is_win(self.board[0]) or self.is_lose() or self.is_draw()

    def switch_player(self, current_player):
        return 'enemy' if current_player == 'me' else 'me'


In [173]:
class TicTacToeEnvironment:
    def __init__(self):
        self.state = State()  # 상태 초기화
        self.board_size = (3, 3)
        self.action_space = range(self.board_size[0] * self.board_size[1])  # 가능한 액션 공간 (9개의 칸)
        self.n_actions = len(self.action_space)

        # 보상 체계: 승리 시 1, 패배 시 -1, 무승부는 0, 진행 중에는 보상 없음
        self.reward = {'win': 1, 'lose': -1, 'draw': 0, 'progress': 0}

    def reset(self):
        # 환경을 초기화하고 초기 상태 반환
        self.state = State()  # 새로운 상태 객체로 초기화
        return self.state

    def step(self, action):
        if action not in self.action_space:
            raise ValueError(f"Invalid action: {action}")

        # 1D action을 2D 좌표로 변환
        row, col = divmod(action, self.board_size[1])

        # 현재 플레이어가 선택한 위치에 말 두기
        if (row, col) not in self.state.available_moves():
            raise ValueError(f"Invalid move: position ({row}, {col}) is already occupied")

        # 다음 상태로 전환
        self.state = self.state.next_state((row, col))

        # 게임 종료 여부 확인
        done = self.state.is_done()

        # 보상 계산
        if self.state.is_win(self.state.board[0]):  # 내가 이긴 경우
            reward = self.reward['win']
        elif self.state.is_lose():  # 내가 진 경우
            reward = self.reward['lose']
        elif self.state.is_draw():  # 무승부인 경우
            reward = self.reward['draw']
        else:
            reward = self.reward['progress']  # 게임이 아직 진행 중인 경우

        return self.state, reward, done

    def render(self):
        # 3x3 보드를 생성하여 빈 칸은 ' ', 플레이어는 'X', AI는 'O'로 채운다
        board = np.full((3, 3), ' ')

        # 플레이어의 움직임('X')와 AI의 움직임('O')을 표시
        for i in range(3):
            for j in range(3):
                if self.state.board[0][i][j] == 1:  # 플레이어의 말 'X'
                    board[i][j] = 'X'
                elif self.state.board[1][i][j] == 1:  # AI의 말 'O'
                    board[i][j] = 'O'

        # 3x3 보드 출력
        for row in board:
            print(" | ".join(row))
            print("-" * 9)

In [174]:
import math
import random

class MCTSNode:
    def __init__(self, state, parent=None, action=None):
        self.state = state  # 현재 노드의 상태
        self.parent = parent  # 부모 노드
        self.action = action  # 부모로부터 이 노드로 오게 한 액션
        self.children = []  # 자식 노드들
        self.visits = 0  # 이 노드가 방문된 횟수
        self.wins = 0  # 이 노드를 통한 승리 횟수

    def is_fully_expanded(self):
        # 자식 노드들이 다 생성되었는지 확인
        return len(self.children) == len(self.state.available_moves())

    def best_child(self, exploration_param=1.41):
        # UCB1 공식을 이용해 가장 가치 있는 자식 노드를 반환
        best_value = -float('inf')
        best_node = None
        for child in self.children:
            # UCB1 계산
            ucb1_value = (child.wins / child.visits) + exploration_param * math.sqrt(math.log(self.visits) / child.visits)
            if ucb1_value > best_value:
                best_value = ucb1_value
                best_node = child
        return best_node

    def expand(self):
        # 자식 노드가 없는 새로운 행동을 선택하고 확장
        available_actions = self.state.available_moves()
        for action in available_actions:
            action_1d = action[0] * 3 + action[1]  # 2D action을 1D로 변환
            if not any(child.action == action_1d for child in self.children):
                # 새로운 자식 노드 생성
                new_state = self.state.next_state(action)
                child_node = MCTSNode(new_state, parent=self, action=action_1d)  # 1D action을 저장
                self.children.append(child_node)
                return child_node

    def simulate(self):
        # 랜덤 시뮬레이션을 통해 승리 여부 확인
        current_simulation_state = self.state
        while not current_simulation_state.is_done():
            available_moves = current_simulation_state.available_moves()
            action = random.choice(available_moves)  # 무작위로 액션 선택
            current_simulation_state = current_simulation_state.next_state(action)

        if current_simulation_state.is_win(current_simulation_state.board[0]):
            return 1  # 승리
        elif current_simulation_state.is_lose():
            return -1  # 패배
        else:
            return 0  # 무승부


    def backpropagate(self, result):
        # 시뮬레이션 결과를 바탕으로 트리를 역방향으로 업데이트
        self.visits += 1
        self.wins += result
        if self.parent:
            self.parent.backpropagate(-result)  # 상대방의 입장에서는 결과를 뒤집어야 함

class MCTSAgent:
    def __init__(self, environment, iterations=1000):
        self.environment = environment  # TicTacToe 환경
        self.iterations = iterations  # 시뮬레이션 반복 횟수

    def select_action(self, state):
        root_node = MCTSNode(state)

        for _ in range(self.iterations):
            # MCTS 4단계: 선택, 확장, 시뮬레이션, 역전파
            node = root_node
            # 1. 선택 단계: 자식 중 가장 좋은 노드를 선택하며 내려감
            while not node.state.is_done() and node.is_fully_expanded():
                node = node.best_child()

            # 2. 확장 단계: 자식 노드 확장
            if not node.state.is_done() and not node.is_fully_expanded():
                node = node.expand()

            # 3. 시뮬레이션 단계: 랜덤하게 게임 끝까지 진행
            result = node.simulate()

            # 4. 역전파 단계: 결과를 부모 노드들로 전달하며 업데이트
            node.backpropagate(result)

        # 가장 많은 승리를 기록한 자식 노드를 선택
        best_child = root_node.best_child(exploration_param=0)
        return best_child.action  # 선택된 행동 반환
